In [5]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages(library(DBI))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(foreach))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))

In [8]:
# Read the meta data
clin_table = "models_clinical_2020-01-14"
sanctumdbcon <- RMariaDB::dbConnect(
    drv = RMariaDB::MariaDB(),
    dbname = "Devdb",
    user = "admin",
    password = "JohnLennon1980",
    host = "sanctum-instance-1.cnklj0yzvfnv.us-east-1.rds.amazonaws.com",
    port = 3306
  )
clin_data <- dbReadTable(sanctumdbcon, name = clin_table)
dbDisconnect(sanctumdbcon)

In [10]:
clin_data

key,model_id,value
Model,1,CTG-0001
TumorGraft ID,1,TGSM-0032
Tumor type,1,Abdominal Desmoid Tumor
Model,2,CTG-0002
TumorGraft ID,2,OCA-416
Tumor type,2,Adenocarcinoma
Model,3,CTG-0003
TumorGraft ID,3,TGSM-0024
Tumor type,3,Adenocarcinoma
Model,4,CTG-0004


In [9]:
# clin_data <- dcast(subset(clin_data, !is.na(key)), 
#                  formula = model_id ~ key,
#                  fun.aggregate = max) %>%
# subset(., Model %in% colnames(counts)) # keep metadata for relavant samples

# colnames(clin_data) <- gsub('..via.TruSight.clinical.', '', colnames(clin_data))
# clin_data[] <- lapply(clin_data, toupper)
# clin_data[clin_data=="PENDING" | clin_data=="NOT AVAILABLE" | clin_data=="OTHER" ] <- NA
# clin_data[] <- lapply(clin_data, function(x) gsub("DIFFERENTIATED", "DIFF.", x))
# clin_data[] <- lapply(clin_data, function(x) gsub("SQUAMOUS CELL", "SC", x))
# # remove (largely) empty columns 
# clin_data <- clin_data[apply(clin_data, 2, function(x) sum(is.na(x)) < nrow(clin_data)/2)]
# # identify numeric columns
# clin_data <- retype(clin_data)

### Proteomics data

In [13]:
counts  <- read.csv("./data/Prot_Data_F20FTSUSAT1137_47AML_2021-03-29.csv")
head(counts, 2)

X,ProteinGroupId,ProteinGroup,Protein,SingleProtein,Sample,Intensity,Counts,ProteinGroupName,hgnc_symbol
1,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2226,17.225,153194,PAL4E_HUMAN,PPIAL4E
2,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2227,18.070,275177,PAL4E_HUMAN,PPIAL4E


In [14]:
### Missingness, filtering and imputation ###
NAs_persamp <- sapply(Filter(is.numeric, counts), function(x) sum(is.na(x)))
# NAs_perfeature <- rowSums(is.na(Filter(is.numeric, counts)))
# prcntNAs_persamp <- round(NAs_persamp/length(NAs_perfeature)*100, 1)
# prcntNAs_perfeature <- round(NAs_perfeature/length(NAs_persamp)*100, 1)

In [15]:
NAs_persamp

X Intensity    Counts 
        0         0         0